# Marathi2Vec Language Modeling

Thanks to NirantK and cstorm125 for the works Hindi2Vec and Thai2Vec, both of which helped me make this notebook. 

The goal of this notebook is to train Marathi word embeddings using the fast.ai version of AWD LSTM with dropouts, the data was fulled from Wikipedia.

A perplexity of was achieved, there has been no comparable research work in Marathi Language at the point of writing. (2nd October, 2018)


In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import dill as pickle
import json 
import re

from IPython.display import Image
from IPython.core.display import HTML

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

In [1]:
from fastai.learner import *
from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

 - Download or clone WikiExtractor from github
 - To run on windows look at [this](https://github.com/attardi/wikiextractor/issues/89#issuecomment-272062219).
 
 Run the following command :
 
 ```python WikiExtractor.py mrwiki-latest-pages-articles.xml -o extract -b 10M --ignored_tags abbr,b,big --discard_elements gallery,timeline,noinclude```

In [ ]:
%%time
%%prun

data = '../extract/'
extracted_files = f'{data}'

def clean_files(extracted_filelist, TRN):    
    cleaned_all = []
    for ext_file in extracted_filelist:
        input_file = f'{EXT}{ext_file}'
        with open(input_file,'r', encoding='utf-8') as f:
            raw_txt = f.readlines()
            cleaned_doc = []
            for line in raw_txt:
                new_line = re.sub('<[^<]+?>', '', line)
                new_line = re.sub('__[^<]+?__', '', new_line) 
                new_line = new_line.strip()
                if new_line != '':
                    cleaned_doc.append(new_line)

            new_doc = "\n".join(cleaned_doc)
            cleaned_all.append(new_doc)
            with open(f"{TRN}{ext_file}.txt", "w", encoding='utf-8') as text_file:
                text_file.write(new_doc)
    return cleaned_all

cleaned_all = clean_files(ext_files, TRN)

